In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Get files

In [2]:
working_directory = 'DATA'

In [3]:
!dir DATA

 Volume in drive C is Local Disk
 Volume Serial Number is FE3C-B042

 Directory of C:\Users\luisf\github\Computer-Vision-Udemy\DATA

05/27/2020  11:08 AM    <DIR>          .
05/27/2020  11:08 AM    <DIR>          ..
05/27/2020  11:08 AM    <DIR>          .ipynb_checkpoints
05/27/2020  11:08 AM           193,288 00-puppy.jpg
05/27/2020  11:08 AM            46,400 bank_note_data.txt
05/27/2020  11:08 AM           500,238 bricks.jpg
05/27/2020  11:08 AM           494,070 cartoon_receipt.jpg
05/27/2020  11:08 AM            22,394 car_plate.jpg
05/27/2020  11:08 AM         1,036,601 chessboard_mat.jpg
05/27/2020  11:08 AM         2,918,981 coins_on_white.jpg
05/27/2020  11:08 AM           533,390 crossword.jpg
05/27/2020  11:08 AM         2,949,661 damien_hirst_dot.jpg
05/27/2020  11:08 AM           127,252 Denis_Mukwege.jpg
05/27/2020  11:08 AM           167,836 dog_backpack.jpg
05/27/2020  11:08 AM           167,836 dog_backpack.png
05/27/2020  11:08 AM             7,627 dot_grid.png
05/2

# Optical Flow

In [2]:
# corner detection
corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(200, 200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

## Lucas Kanade Method

In [9]:
capture = cv2.VideoCapture(0)

ret, prev_frame = capture.read()

prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Points to track
prev_points = cv2.goodFeaturesToTrack(prev_frame_gray, mask=None, **corner_track_params)

# for visualization (drawing on later)
mask = np.zeros_like(prev_frame)

while True:
    ret, frame = capture.read(0)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    next_points, status, error = cv2.calcOpticalFlowPyrLK(prev_frame_gray, frame_gray, prev_points, None, **lk_params)
    
    good_new_points = next_points[status==1]
    good_prev_points = prev_points[status==1]
    
    for i, (new_p, prev_p) in enumerate(zip(good_new_points, good_prev_points)):
        x_new, y_new = new_p.ravel()
        x_prev, y_prev = prev_p.ravel()
        
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        
        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
        
    img = cv2.add(frame, mask)
    cv2.imshow('tracking', img)
    
    k = cv2.waitKey(30) & 0xFF
    if k==27:
        break
        
    prev_frame_gray = frame_gray.copy()
    prev_points = good_new_points.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
capture.release()

## Dense Optical Flow

In [8]:
capture = cv2.VideoCapture(0)

ret, prev_frame = capture.read()

prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)


# hsv mask (Hue Saturation Value)
hsv_mask = np.zeros_like(prev_frame)
hsv_mask[:, :, 1] = 255

while True:
    ret, frame = capture.read(0)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_frame_gray,frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    #conver to polar coordinates
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)
    
    # operation to identify a certain direction better
    hsv_mask[:,:,0] = ang/2
    # normalize values bw 0 and 255
    hsv_mask[:,:,2]= cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
    img = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow('frame', img)
    
    k = cv2.waitKey(30) & 0xFF
    if k==27:
        break
        
    prev_frame_gray = frame_gray.copy()
    
cv2.destroyAllWindows()
capture.release()

# MeanShift and CamShift tracking

In [7]:
classifier = cv2.CascadeClassifier(working_directory+'/haarcascades/haarcascade_frontalface_default.xml')

In [8]:
def detect_face(img):
    face_img = img.copy()
    face_positions = classifier.detectMultiScale(face_img)
    tuple(face_positions[0])
    return tuple(face_positions[0])

In [17]:
capture = cv2.VideoCapture(0)

ret, frame = capture.read()

(face_x, face_y, width, height) = detect_face(frame)
track_window = (face_x, face_y, width, height)

roi = frame[face_y:face_y+height, face_x:face_x+width]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# calculate color histogram for image
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])

cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX) # normalize histogram array values

# set criteria
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = capture.read(0)
    
    if ret:
        # image to hsv
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Back Projection is a way of recording how well the pixels of a given image fit the distribution of pixels in a histogram model
        dst = cv2.calcBackProject([hsv_frame], [0], roi_hist, [0, 180], 1)
        
        # perform mean shift
        ret, track_window = cv2.CamShift(dst, track_window, criteria)
        
        points = cv2.boxPoints(ret)
        points = np.int0(points)
        
        new_image = cv2.polylines(frame, [points], True, (0, 0, 255), 5)
        
        cv2.imshow('img', new_image)
    
        k = cv2.waitKey(1) & 0xFF
        if k==27:
            break
    else:
        break
    
cv2.destroyAllWindows()
capture.release()